# Structure learning with cause2e
This notebook shows how ```cause2e``` can be used for learning causal graphs. Structure learning (also called causal discovery) can be performed by the ```discovery.StructureLearner``` after reading data and specifying domain knowledge. If we only want to perform a quick exploratory search, we can use the provided reasonable default parameters for the search procedure. However, if we need to finetune the settings for the causal search after encountering problems with the default option, this notebooks shows how to do it. The search is mostly based on the ```py-causal``` package, a wrapper around the well-known JAVA ```TETRAD``` software. ```Cause2e``` aims to use ```py-causal``` algorithms only for the search itself, in order to spare the user from dealing with JAVA error messages when dealing with peripheral tasks.

### Imports

In [ ]:
import os
from cause2e import path_mgr, discovery, knowledge

## Set up paths to data and output directories
This step is conveniently handled by the ```PathManager``` class, which avoids having to wrestle with paths throughout the multistep causal analysis. If we want to perform the analysis in a directory ```'dirname'``` that contains ```'dirname/data'``` and ```'dirname/output'``` as subdirectories, we can also use ```PathManagerQuick``` for an even easier setup. The experiment name is used for generating output files with meaningful names, in case we want to study multiple scenarios (e.g. with varying model parameters). For this analysis, we use the sprinkler dataset.

In [ ]:
cwd = os.getcwd()
wd = os.path.dirname(cwd)
paths = path_mgr.PathManagerQuick(experiment_name='sprinkler',
                                  data_name='sprinkler.csv',
                                  directory=wd
                                  )

## Initialize the StructureLearner
As in the other notebooks, we set up a ```StructureLearner``` and read our data.

In [ ]:
learner = discovery.StructureLearner(paths)
learner.read_csv(index_col=0)

The first step in the analysis should be an assessment of which variables we are dealing with. In the sprinkler dataset, each sample tells us 
- the current season
- whether it is raining
- whether our lawn sprinkler is activated
- whether our lawn is slippery
- whether our lawn is wet.

In [ ]:
print(learner.variables)

It necessary to communicate to the ```StructureLearner``` if the variables are discrete, continuous, or both. We check how many unique values each variable takes on in our sample and deduce that all variables are discrete.

In [ ]:
print(learner.data.nunique())

This information is passed to the ```StructureLearner``` by indicating the exact sets of discrete and continuous variables.

In [ ]:
learner.discrete = set(learner.variables)
learner.continuous = set()

### Provide domain knowledge
Humans can often infer parts of the causal graph from domain knowledge. The nodes are always just the variables in the data, so the problem of finding the right graph comes down to selecting the right edges between them.

As a reminder: The correct causal graph has an edge from variable A to variable B if and only if variable A directly influences variable B (changing the value of variable A changes the value of variable B if we keep all other variables fixed).


There are three ways of passing domain knowledge:
- Indicate which edges must be present in the causal graph.
- Indicate which edges must not be present in the causal graph.
- Indicate a temporal order in which the variables have been created. This is then used to generate forbidden edges, since the future can never influence the past.

In this example, we only assume that the current season is directly influencing the weather and the probability that the sprinkler is on. This makes sense: During the summer, it is less likely to rain and sprinklers are more likely to be activated.

In [ ]:
required = {('Season', 'Rain'), ('Season', 'Sprinkler')}
edge_creator = knowledge.EdgeCreator()
edge_creator.forbid_edges(required)

We pass the knowledge to the ```StructureLearner``` and check if it has been correctly received.

In [ ]:
learner.set_knowledge(edge_creator)
print(learner.knowledge)

## Select and use a structure learning algorithm
Now that the ```StructureLearner``` has received the data and the domain knowledge, we can try to recover the original graph using causal discovery methods provided by the internally called ```py-causal``` package. There are many parameters that can be tuned (choice of algorithm, search score, independence test, hyperparameters, ...) and we can get an overview by calling some informative methods of the learner.

In [ ]:
learner.show_search_algos()
learner.show_search_scores()
learner.show_independence_tests()

To make an informed choice, we can browse through the proposed search algorithms and decide which one fits our problem. Let us have a look at the FGES algorithm, which is a well known score-based algorithm that is suitable for a mix of continuous and discrete data. Note that it also accepts domain knowledge, which makes it a good starting point for many datasets.

In [ ]:
learner.show_algo_info('fges')

The description tells us that we can select a search score and pass our domain knowledge. If we actually want to call the algorithm, we need to know if it requires additional hyperparameters and what they mean. These can be inspected via another utility method. Since FGES requires a score, we need to pass one to ```show_algo_params```, but it seems that the choice does not affect the output, so we just choose one at random from the above list.

In [ ]:
learner.show_algo_params('fges', score_name='cg-bic-score')

Let us try out a possible search configuration.

In [ ]:
learner.run_search(algo='fges', scoreId='cg-bic-score', maxDegree=5, faithfulnessAssumed=True, symmetricFirstStep=True)

The output of the search is a proposed causal graph. We can ignore the warning about stopping the Java Virtual Machine (needed by ```py-causal``` which is a wrapper around the ```TETRAD``` software that is written in Java) if we do not run into any problems. If the algorithm cannot orient all edges, we need to do this manually. Therefore, the output includes a list of all undirected edges, so we do not miss them in complicated graphs with many variables and edges. In our case, all the edges are already oriented.

The result seems reasonable:
- The weather depends on the season.
- The sprinkler use also depends on the season.
- The lawn will be wet if it rains or if the sprinkler is activated.
- The lawn will be slippery if it is wet.

We can also see that the result is automatically saved to different file formats and that our graph respects the previously indicated domain knowledge.

In order to spare users from the pain of going through all the above reading whenever they want to perform just a quick exploratory analysis, we have provided the above configuration as default arguments (FGES with CG-BIC score for possibly mixed datatypes, respecting domain knowledge, assuming faithfulness, using symmetric first step) that let us start the search without any finetuning. Just call ```run_quick_search()``` and you are good to go.

In [ ]:
learner.run_quick_search()

In this notebook we want to show how to switch to a different algorithm, e.g. a variant of the constraint-based PC algorithm, which can be found under the name ```pc-all``` in the above listing of algorithms. The procedure is the same as above, it just requires some reading of ```py-causal```'s algorithm descriptions.

In [ ]:
learner.show_algo_info('pc-all')

In this case, we need to pass a ```test_name``` to ```show_algo_params``` instead of a ```score_name```. Again, we pick one at random since it does not seem to change the description.

In [ ]:
learner.show_algo_params('pc-all', test_name='bdeu-test')

Now that we know possible configuration options, we can select a few at random and check the result.

In [ ]:
learner.run_search(algo='pc-all', stableFAS=True, conflictRule=1, save_graph=False)

The output is worse than the one with FGES, but the situation might be reversed for a different problem, so having the ability to quickly switch between algorithms and hyperparameters is a handy tool. Some algorithms are only suited for certain types of data, some cannot accept domain knowledge, some produce outputs that differ from the mixed graph format (other options such as PAG are currently not supported by ```cause2e```'s graph handling). Feel free to play around with different algorithms and configurations to explore the possibilities of causal discovery!